In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

tf.enable_eager_execution()

print("Tensor flow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

### Download the data

In [ ]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(
    fname = os.path.basename(train_dataset_url),
    origin = train_dataset_url
)

In [ ]:
print("Local copy of dataset: {}".format(train_dataset_fp))

### Inspect the data

In [ ]:
!head -n5 {train_dataset_fp}

Labels description
0: iris setosa
1: iris versicolor
2: iris virginica

In [ ]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Labels: {}".format(label_name))

### Create a tf.data.Dataset

In [ ]:
batch_size = 32
train_dataset = tf.contrib.data.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names = column_names,
    label_name = label_name,
    num_epochs = 1
)

In [ ]:
features, labels = next(iter(train_dataset))
features

In [ ]:
plt.scatter(
    features['petal_length'].numpy(),
    features['sepal_length'].numpy(),
    c = labels.numpy(),
    cmap = 'viridis'
)

plt.xlabel('Petal Length')
plt.ylabel('Sepal Length')
plt.show()

In [ ]:
def pack_feature_vector(features, labels):
    """Pack features into a single array"""
    features = tf.stack(list(features.values()), axis = 1)
    return features, labels

In [ ]:
# Pack the features pairs of each (features, label) into the training dataset
train_dataset = train_dataset.map(pack_feature_vector)

# Look at the features dataset array that's now an array of shape (batch_size, num_features)
features, labels = next(iter(train_dataset))
print(features[:5])

# Model selection and training

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape = (4,)),
    tf.keras.layers.Dense(10, activation = tf.nn.relu),
    tf.keras.layers.Dense(3)
])

In [ ]:
predictions = model(features)
predictions[:5]

In [ ]:
# Convert the logits to a probability for each class
tf.nn.softmax(predictions[:5])
print("Predictions: {}".format(tf.argmax(predictions, axis = 1)))
print("Labels: {}".format(labels))

### Define the Loss Gradient Function

In [ ]:
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

In [ ]:
l = loss(model, features, labels)
print("Loss test: {}".format(l))

In [ ]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

# Create the Optimizer

In [ ]:
# The following implements Stochastic Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
global_step = tf.Variable(0)
loss_value, grads = grad(model, features, labels)
print("Step: {}, Initial loss: {}".format(global_step.numpy(), loss_value.numpy()))
optimizer.apply_gradients(zip(grads, model.trainablevariables, global_step))
print("Step: {}, Loss: {}".format(global_step.numpy(), loss(model, features, labels).numpy()))

## Training Loop

In [ ]:
import tensorflow as tf

# Keep the plotting results
training_loss_results = []
training_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tf.metrics.Mean()
    epoch_accuracy = tf.metrics.Accuracy()
    
    # Training loop with batches of 32
    for x, y in train_dataset:
        # Optimize the model
        loss_value = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables), global_step)
        # Track progress
        epoch_loss_avg(loss_value) # add current batch loss
        # Compare predicted label to actual label
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)
    # end epoch
    training_loss_results.append(epoch_loss_avg.result())
    training_accuracy_results.append(epoch_accuracy.result())
    
    if epoch % 50 == 0:
        print("Epoch: {:03d}, Loss:{:.3f}, Accuracy: {:3%}".format(
            epoch,
            epoch_loss_avg.result(),
            epoch_accuracy.result()
        ))

### Visualize the Loss Function over time

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

## Evaluate the model's effectiveness

## Set up the test dataset

In [ ]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url), origin=test_url)
test_dataset = tf.data.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False
)

test_dataset = test_dataset.map(pack_features_vector)

## Evaluate the Model on a test Dataset

In [ ]:
test_accuracy = tf.metrics.Accuracy()

for (x, y) in test_dataset:
    logits = model(x)
    prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
    
print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

# Looking at the last batch we can see the model is usually correct
tf.stack([y, prediction], axis=1)

## Use the model to make predictions

In [ ]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    p = tf.nn.softmax(logits)[class_idx]
    name = class_names[class_idx]
    print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))